Funktioniert!

In [11]:

import cv2
import matplotlib.pyplot as plt
import numpy as np
import skimage
from skimage import  metrics
# Load the input image

def evaluate_image_quality(img):
    # calculate the structural similarity index between the original and modified images
    ssim = metrics.structural_similarity(original_img, img, multichannel=True)
    return ssim



original_img = cv2.imread('/home/matz/SynologyDrive/FHNW/bver/CODE/bver_projekt/bver_projekt/Aufnahmen_FHNW/bild2_Color.png')
img = original_img
best_alpha = 1.0
best_beta = 0
best_quality = -1

# Loop through a range of alpha and beta values
for alpha in np.arange(0.5, 1.5, 0.1):
    for beta in np.arange(-50, 50, 10):
        # Apply the contrast change
        img_temp = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
        # evaluate the result
        quality = evaluate_image_quality(img_temp)
        # check if it is better than the current best
        if quality > best_quality:
            best_alpha = alpha
            best_beta = beta
            best_quality = quality
         
  
for i in range(1,13):
    a= str(i)
    img = cv2.imread('/home/matz/SynologyDrive/FHNW/bver/CODE/bver_projekt/bver_projekt/Aufnahmen_FHNW/bild'+a+'_Color.png')      
            
            

    # Use the best alpha and beta values
    img = cv2.convertScaleAbs(img, alpha=best_alpha, beta=best_beta)


    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


    # Use the Canny edge detector to find edges in the image
    edges = cv2.Canny(gray, 50, 150, apertureSize = 3)



    # Compute the threshold for the image
    #ret, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)

    # Find the contours in the image
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    door = 0
    # Loop through the contours and draw rectangles around the doorframes
    for i, (contour, h) in enumerate(zip(contours, hierarchy[0])):
        # If the contour does not have a parent, draw it on the image
        
        '''      x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)  '''
        if h[3] == -1:
            # Find the bounding rectangle for the contour
            x, y, w, h = cv2.boundingRect(contour)
            if h/w >1.75 and h> 350 and w>250:
                # Draw the rectangle on the image
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                door =1
                print("Door Detected")  
            
        
    if door == 0:
        print("No Door Detected") 
    # Save the output image
    cv2.imwrite("output"+a+".jpg",img)
    cv2.imwrite("edge"+a+".jpg",edges)
    i=i+1




# Display the resized image
cv2.imshow('hallo', img)

cv2.waitKey(0)
cv2.destroyAllWindows()

/tmp/ipykernel_23682/3227655598.py:10: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim = metrics.structural_similarity(original_img, img, multichannel=True)


No Door Detected
Door Detected
No Door Detected
No Door Detected
Door Detected
No Door Detected
Door Detected
Door Detected
No Door Detected
No Door Detected
Door Detected
No Door Detected
No Door Detected


In [3]:
# Import the necessary libraries
import cv2
import numpy as np

# Read in the image
img = cv2.imread('/home/matz/SynologyDrive/FHNW/bver/CODE/bver_projekt/bver_projekt/door1_Color.png')
# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Use the Canny edge detector to find edges in the image
edges = cv2.Canny(gray, 50, 150, apertureSize = 3)

# Use the Hough transform to detect lines in the image
line_vert = cv2.HoughLines(edges, 1, np.pi/180, 275)

line_hor = cv2.HoughLines(edges,1,np.pi/180, 250)
# Create a black image to draw the lines on
line_img = np.zeros_like(img)


# Loop through the lines and draw them on the image
for line in line_vert:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

for line in line_hor:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    
    cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2) 
    

result = cv2.bitwise_and(img, line_img)


# Show the image with the lines drawn on it
cv2.imshow('Image with lines', img)
#cv2.imshow('Image with lines', edges)
cv2.waitKey(0)
cv2.destroyAllWindows()



''' # define a function that takes in the coordinates of two points on a line and returns the slope of the line
def get_slope(x1, y1, x2, y2):
    return (y2 - y1) / (x2 - x1)

# define a function that takes in the coordinates of two points on a line and returns the y-intercept of the line
def get_y_intercept(x, y, slope):
    return y - (slope * x)

# define a function that takes in the coordinates of two points on a line and returns the equation of the line in the form "y = mx + b"
def get_line_equation(x1, y1, x2, y2):
    slope = get_slope(x1, y1, x2, y2)
    y_intercept = get_y_intercept(x1, y1, slope)
    return f"y = {slope}x + {y_intercept}"

# define a function that takes in the equations of two lines and checks if they intersect
def lines_intersect(line1, line2):
    # if the slopes of the two lines are the same, they are parallel and do not intersect
    if line1[0] == line2[0]:
        return False
    else:
        # if the slopes are different, the lines intersect at some point
        return True

# define the coordinates of two points on each line
line1_point1 = (1, 1)
line1_point2 = (2, 2)

line2_point1 = (3, 3)
line2_point2 = (4, 4)

# get the equations of the lines
line1_equation = get_line_equation(line1_point1[0], line1_point1[1], line1_point2[0], line1_point2[1])
line2_equation = get_line_equation(line2_point1[0], line2_point1[1], line2_point2[0], line2_point2[1])

# check if the lines intersect
if lines_intersect(line1_equation, linecv2.drawContours(img, [contour], -1, (0, 255, 0), 2)2_equation):
    print("The lines intersect.")
else:
    print("The lines do not intersect.")
  '''

''' This program first defines several helper functions:

    get_slope() calculates the slope of a line given the coordinates of two points on the line.
    get_y_intercept() calculates the y-intercept of a line given the coordinates of a point on the line and the slope of the line.
    get_line_equation() calculates the equation of a line in the form "y = mx + b" given the coordinates of two points on the line.
    lines_intersect() takes in the equations of two lines and checks if they intersect by comparing their slopes.

It then uses these helper functions to compare two lines and check if they intersect. It defines the coordinates of two points on each line, calculates the equations of the lines, and then uses the lines_intersect() function to check if the lines intersect.
 '''

' This program first defines several helper functions:\n\n    get_slope() calculates the slope of a line given the coordinates of two points on the line.\n    get_y_intercept() calculates the y-intercept of a line given the coordinates of a point on the line and the slope of the line.\n    get_line_equation() calculates the equation of a line in the form "y = mx + b" given the coordinates of two points on the line.\n    lines_intersect() takes in the equations of two lines and checks if they intersect by comparing their slopes.\n\nIt then uses these helper functions to compare two lines and check if they intersect. It defines the coordinates of two points on each line, calculates the equations of the lines, and then uses the lines_intersect() function to check if the lines intersect.\n '